In [1]:
import os
from sklearn.model_selection import train_test_split
import numpy as np

In [2]:
import os 
os.chdir('/home/meiyih/Attention-Guided-Low-light-Image-Enhancement-with-Scene-Text-Restoration')

In [3]:
import os
import numpy as np
import cv2
from skimage import exposure

# dark_img_path = './dataset/Sony/short/00001_00_0.033s.png'

# rgb = cv2.imread(dark_img_path, cv2.COLOR_BGR2RGB) * 150
# img = np.uint8(np.minimum(rgb, 255.0))
# cv2.imwrite('Sony_expose_example/dark_img_exposed150.png', img)


# dark_img_path = './dataset/Sony/short/00001_00_0.04s.png'

# rgb = cv2.imread(dark_img_path, cv2.COLOR_BGR2RGB) * 50
# img = np.uint8(np.minimum(rgb, 255.0))
# cv2.imwrite('Sony_expose_example/dark_img_exposed50.png', img)


dark_img_path = './dataset/SID/Sony/test/short/10054_00_0.1s.png'

rgb = cv2.imread(dark_img_path, cv2.COLOR_BGR2RGB) * 50
img = np.uint8(np.minimum(rgb, 255.0))
cv2.imwrite('dark_img_exposed.png', img)

True

In [2]:
DATASET_PATH = '/home/meiyih/BiSeNet_text_seg/datasets'

imgs = os.listdir(os.path.join(DATASET_PATH, 'masks'))
train, val = train_test_split(imgs, train_size=0.8)

In [3]:
X_train = np.c_[list(map(lambda x: os.path.join('syn_1.0-2.0_low_input', x), train))]
Y_train = np.c_[list(map(lambda x: os.path.join('masks', x), train))]

X_val = np.c_[list(map(lambda x: os.path.join('syn_1.0-2.0_low_input', x), val))]
Y_val = np.c_[list(map(lambda x: os.path.join('masks', x), val))]

In [4]:
train_annotation = '\n'.join(list(map(lambda x: ','.join(x), np.c_[X_train, Y_train])))
val_annotation = '\n'.join(list(map(lambda x: ','.join(x), np.c_[X_val, Y_val])))

if not os.path.isdir('/home/meiyih/BiSeNet_text_seg/datasets/customDataset'):
  os.mkdir('/home/meiyih/BiSeNet_text_seg/datasets/customDataset')

with open('/home/meiyih/BiSeNet_text_seg/datasets/customDataset/train.txt', 'w') as f:
  f.write(train_annotation)

with open('/home/meiyih/BiSeNet_text_seg/datasets/customDataset/val.txt', 'w') as f:
  f.write(val_annotation)

In [5]:
cfg = """cfg = dict(
    model_type='bisenetv1',
    n_cats=2,
    num_aux_heads=2,
    lr_start=1e-2,
    weight_decay=5e-4,
    warmup_iters=1000,
    max_iter=80000,
    dataset='CustomDataset',
    im_root='/media/meiyih/meiyih_datasets/BiSeNet_text_seg/datasets',
    train_im_anns='./datasets/customDataset/train.txt',
    val_im_anns='./datasets/customDataset/val.txt',
    scales=[0.75, 2.],
    cropsize=[1024, 1024],
    eval_crop=[1024, 1024],
    eval_scales=[0.5, 0.75, 1.0, 1.25, 1.5, 1.75],
    ims_per_gpu=8,
    eval_ims_per_gpu=2,
    use_fp16=True,
    use_sync_bn=False,
    respth='./res',
)"""

with open('/home/meiyih/BiSeNet_text_seg/configs/bisenetv1_customDataset.py', 'w') as f:
  f.write(cfg)

In [7]:
customDataset = '''#!/usr/bin/python
# -*- encoding: utf-8 -*-

import os
import os.path as osp
import json

import torch
from torch.utils.data import Dataset, DataLoader
import torch.distributed as dist
import cv2
import numpy as np

import lib.transform_cv2 as T
from lib.sampler import RepeatedDistSampler
from lib.base_dataset import BaseDataset

labels_info =[
{'name':"background","id":0,"color":[0,0,0],"trainId":0},
{'name':"target","id":1,"color":[255,255,255],"trainId":1}
]

class CustomDataset(BaseDataset):
    def __init__(self, dataroot, annpath, trans_func=None, mode='train'):
        super(CustomDataset, self).__init__(
                dataroot, annpath, trans_func, mode)
        self.n_cats = 2
        self.lb_ignore = 255
        self.lb_map = np.arange(256).astype(np.uint8)
        for el in labels_info:
            self.lb_map[el['id']] = el['trainId']

        self.to_tensor = T.ToTensor(
            mean=(0.3257, 0.3690, 0.3223), # city, rgb
            std=(0.2112, 0.2148, 0.2115),
        )'''

with open('/home/meiyih/BiSeNet_text_seg/lib/customDataset.py', 'w') as f:
  f.write(customDataset)

In [11]:
%cd BiSeNet_text_seg

/home/meiyih/BiSeNet_text_seg


In [12]:
!python -m torch.distributed.launch --nproc_per_node=1 tools/train_amp.py --config configs/bisnetv1_customDataset.py 

Traceback (most recent call last):
  File "/home/meiyih/BiSeNet_text_seg/tools/train_amp.py", line 53, in <module>
    cfg = set_cfg_from_file(args.config)
  File "/home/meiyih/BiSeNet_text_seg/./configs/__init__.py", line 15, in set_cfg_from_file
    spec_loader = spec.loader.exec_module(cfg_file)
  File "<frozen importlib._bootstrap_external>", line 846, in exec_module
  File "<frozen importlib._bootstrap_external>", line 982, in get_code
  File "<frozen importlib._bootstrap_external>", line 1039, in get_data
FileNotFoundError: [Errno 2] No such file or directory: 'configs/bisnetv1_customDataset.py'
Killing subprocess 103293
Traceback (most recent call last):
  File "/home/meiyih/anaconda3/envs/pix_clone2/lib/python3.9/runpy.py", line 197, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/home/meiyih/anaconda3/envs/pix_clone2/lib/python3.9/runpy.py", line 87, in _run_code
    exec(code, run_globals)
  File "/home/meiyih/anaconda3/envs/pix_clone2/lib/pytho

In [ ]:
#!/usr/bin/python
# -*- encoding: utf-8 -*-

import sys
sys.path.insert(0, '.')
import os
import os.path as osp
import random
import logging
import time
import argparse
import numpy as np
from tabulate import tabulate

import torch
import torch.nn as nn
import torch.distributed as dist
from torch.utils.data import DataLoader
import torch.cuda.amp as amp
import torchvision
from lib.models import model_factory
from configs import set_cfg_from_file
from lib.get_dataloader import get_data_loader
from evaluate import eval_model
from lib.ohem_ce_loss import OhemCELoss
from lib.lr_scheduler import WarmupPolyLrScheduler
from lib.meters import TimeMeter, AvgMeter
from lib.logger import setup_logger, print_log_msg
from pytorchtools import EarlyStopping

from pytorch_msssim import ssim, ms_ssim

## fix all random seeds
#  torch.manual_seed(123)
#  torch.cuda.manual_seed(123)
#  np.random.seed(123)
#  random.seed(123)
#  torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = True
#  torch.multiprocessing.set_sharing_strategy('file_system')

#torchrun --standalone --nnodes=1 --nproc_per_node=1 tools/train_amp.py  YOUR_TRAINING_SCRIPT.py (--config configs/bisenetv1_customDataset.py)


def parse_args():
    parse = argparse.ArgumentParser()
    parse.add_argument('--local_rank', dest='local_rank', type=int, default=-1,)
    parse.add_argument('--port', dest='port', type=int, default=44557,)
    parse.add_argument('--config', dest='config', type=str,
            default='configs/bisenetv2.py',)
    parse.add_argument('--finetune-from', type=str, default=None,)
    return parse.parse_args()

args = parse_args()
cfg = set_cfg_from_file(args.config)



def MAELoss(out_image, gt_image):
    return torch.mean(torch.abs(out_image - gt_image))
    
def MS_SSIMLoss(out_image, gt_image):
    return 1 - ms_ssim(out_image, gt_image, data_range=1, size_average=True)    

def set_model():
    logger = logging.getLogger()
    net = model_factory[cfg.model_type](cfg.n_cats)
    if not args.finetune_from is None:
        logger.info(f'load pretrained weights from {args.finetune_from}')
        net.load_state_dict(torch.load(args.finetune_from, map_location='cpu'))
    if cfg.use_sync_bn: net = nn.SyncBatchNorm.convert_sync_batchnorm(net)
    net.cuda()
    net.train()
    #criteria_pre = MAELoss()
    #criteria_aux = [MS_SSIMLoss() for _ in range(cfg.num_aux_heads)]
    return net#, criteria_pre, criteria_aux


def set_optimizer(model):
    if hasattr(model, 'get_params'):
        wd_params, nowd_params, lr_mul_wd_params, lr_mul_nowd_params = model.get_params()
        #  wd_val = cfg.weight_decay
        wd_val = 0
        params_list = [
            {'params': wd_params, },
            {'params': nowd_params, 'weight_decay': wd_val},
            {'params': lr_mul_wd_params, 'lr': cfg.lr_start * 10},
            {'params': lr_mul_nowd_params, 'weight_decay': wd_val, 'lr': cfg.lr_start * 10},
        ]
    else:
        wd_params, non_wd_params = [], []
        for name, param in model.named_parameters():
            if param.dim() == 1:
                non_wd_params.append(param)
            elif param.dim() == 2 or param.dim() == 4:
                wd_params.append(param)
        params_list = [
            {'params': wd_params, },
            {'params': non_wd_params, 'weight_decay': 0},
        ]
    optim = torch.optim.SGD(
        params_list,
        lr=cfg.lr_start,
        momentum=0.9,
        weight_decay=cfg.weight_decay,
    )
    return optim


def set_model_dist(net):
    local_rank = dist.get_rank()
    #local_rank = int(os.environ["LOCAL_RANK"])
    net = nn.parallel.DistributedDataParallel(
        net,
        device_ids=[local_rank, ],
        #  find_unused_parameters=True,
        output_device=local_rank
        )
    return net


def set_meters():
    time_meter = TimeMeter(cfg.max_iter)
    loss_meter = AvgMeter('loss')
    loss_pre_meter = AvgMeter('loss_prem')
    loss_aux_meters = [AvgMeter('loss_aux{}'.format(i))
            for i in range(cfg.num_aux_heads)]
    return time_meter, loss_meter, loss_pre_meter, loss_aux_meters



def train():
    logger = logging.getLogger()
    is_dist = dist.is_initialized()

    ## dataset
    dl = get_data_loader(cfg, mode='train', distributed=is_dist)

    ## model
    net = set_model()

    ## optimizer
    optim = set_optimizer(net)

    ## mixed precision training
    scaler = amp.GradScaler()

    ## ddp training
    net = set_model_dist(net)

    ## meters
    time_meter, loss_meter, loss_pre_meter, loss_aux_meters = set_meters()

    ## lr scheduler
    #lr_schdr = WarmupPolyLrScheduler(optim, power=0.9,
        #max_iter=cfg.max_iter, warmup_iter=cfg.warmup_iters,
        #warmup_ratio=0.1, warmup='exp', last_epoch=-1,)
        
        
        # Define Scheduler
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optim, factor=0.1, patience=10, verbose=True)    
    
    # Define early_stopping
    early_stopping = EarlyStopping(patience=20, verbose=True)
    
    step=0
    logit_hist=[]
    logit0_hist=[]
    logit1_hist=[]

    ## train loop
    for it, (im, lb) in enumerate(dl):
        losses = []
        im = im.cuda()
        lb = lb.cuda()
        #print('im shape : ',im.shape,'lb shape : ',lb.shape)
	
        optim.zero_grad()
        with amp.autocast(enabled=cfg.use_fp16):
            
            logits, *logits_aux = net(im)
            #print(f'dtype of logits : {logits.dtype} , dtype of logits_aux[0] : {logits_aux[0].dtype},dtype of logits_aux[1] : {logits_aux[1].dtype}, dtype of img :{im.dtype}, dtype of rg :{lb.dtype}')
            #print('net output shape : ',logits.shape)
            #print(f'.max() of logits : {logits.max()} , dtype of logits_aux[0] : {logits_aux[0].max()},dtype of logits_aux[1] : {logits_aux[1].max()}, dtype of img :{im.max()}, dtype of rg :{lb.max()}')
            #print(f'.min() of logits : {logits.min()} , dtype of logits_aux[0] : {logits_aux[0].min()},dtype of logits_aux[1] : {logits_aux[1].min()}, dtype of img :{im.min()}, dtype of rg :{lb.min()}')
            #print('net output shape : ',logits.shape)
            #logits=torch.div(logits, 255)
            #logits=torch.clamp(logits, min=0.0, max=1.0)
            #logits_aux[0]=torch.div(logits_aux[0], 255)
            #logits_aux[0]=torch.clamp(logits_aux[0], min=0.0, max=1.0)
            #logits_aux[1]=torch.div(logits_aux[1], 255)
            #logits_aux[1]=torch.clamp(logits_aux[1], min=0.0, max=1.0)
            #print(f'after ............max() of logits : {logits.max()} , dtype of logits_aux[0] : {logits_aux[0].max()},dtype of logits_aux[1] : {logits_aux[1].max()}, dtype of img :{im.max()}, dtype of rg :{lb.max()}')
            #print(f'after .............min() of logits : {logits.min()} , dtype of logits_aux[0] : {logits_aux[0].min()},dtype of logits_aux[1] : {logits_aux[1].min()}, dtype of img :{im.min()}, dtype of rg :{lb.min()}')

            logits= logits.type_as(lb)
            logits_aux[0]=logits_aux[0].type_as(lb)
            logits_aux[1]=logits_aux[1].type_as(lb)
        
            #logits=torch.nn.functional.sigmoid(logits)
            #logits_aux[0]=torch.nn.functional.sigmoid(logits_aux[0])
            #logits_aux[1]=torch.nn.functional.sigmoid(logits_aux[1])
            loss_pre = MAELoss(logits, lb)
            
            #loss_aux1 = MS_SSIMLoss(logits_aux[0], lb)
            loss_aux1 = MAELoss(logits_aux[0], lb)
            loss_aux2 = MAELoss(logits_aux[1], lb)
            loss_aux=[loss_aux1,loss_aux2]
            loss = 1.005*loss_pre + 0.555*sum(loss_aux)
            losses.append(loss.item())
        scaler.scale(loss).backward()
        scaler.step(optim)
        scaler.update()
        torch.cuda.synchronize()

        time_meter.update()
        loss_meter.update(loss.item())
        loss_pre_meter.update(loss_pre.item())
        _ = [mter.update(lss.item()) for mter, lss in zip(loss_aux_meters, loss_aux)]


        ## print training log message #end of one epoch is 125
        if (it + 1) % 125 == 0: #end of 1 epoch 
            #lr = lr_schdr.get_lr()
            mean_loss = sum(losses) / len(losses)
            scheduler.step(mean_loss)
            lr = optim.param_groups[0]['lr']
            print('epoch :',((it+1)/125) , '/n lr is :' ,(optim.param_groups[0]['lr']) , '/n mean loss is : ', (mean_loss))
            print_log_msg(it, cfg.max_iter, lr, time_meter, loss_meter,loss_pre_meter, loss_aux_meters)
            #save some output imges
            
            torchvision.utils.save_image(im , f"./saved_images/low_light_img_{step}.png")       
            torchvision.utils.save_image(lb , f"./saved_images/rgb_img_{step}.png")
            torchvision.utils.save_image(logits , f"./saved_images/rgb_pred_{step}.png")
            
            # early_stopping needs the validation loss to check if it has decresed, 
            # and if it has, it will make a checkpoint of the current model
            early_stopping(mean_loss, net)
            
            if early_stopping.early_stop:
            	print("Early stopping")

            	save_pth = osp.join(cfg.respth, 'model_final.pth')
            	logger.info('\nsave models to {}'.format(save_pth))
            	state = net.module.state_dict()
            	if dist.get_rank() == 0: torch.save(state, save_pth)

            	logger.info('\nevaluating the final model')
            	torch.cuda.empty_cache()
            	heads, mious = eval_model(cfg, net.module)
            	logger.info(tabulate([mious, ], headers=heads, tablefmt='orgtbl'))
            	
            	Path(dir_checkpoint).mkdir(parents=True, exist_ok=True)
            	torch.save(net.state_dict(), str(dir_checkpoint / 'real_checkpoint_epoch{}.pth'.format(((it+1)/250))))
            	logging.info(f'Checkpoint {((it+1)/250)} saved!')
            	
    
            	break #early stopping applied
        
            
            step+=1
        
        if (it + 1) % 125 == 0: #end of 1 epoch 
            logit_hist=[]        
                ## save the model and evaluate the result
        if (it)% 6250 == 0:            
            save_pth = osp.join(cfg.respth, 'model_final'+str(it)+'.pth')
            logger.info('\nsave models to {}'.format(save_pth))
            state = net.module.state_dict()
            if dist.get_rank() == 0: torch.save(state, save_pth)
            
    ## dump the final model and evaluate the result
    save_pth = osp.join(cfg.respth, 'model_the_final.pth')
    logger.info('\nsave models to {}'.format(save_pth))
    state = net.module.state_dict()
    if dist.get_rank() == 0: torch.save(state, save_pth)

    logger.info('\nevaluating the final model')
    torch.cuda.empty_cache()
    heads, mious = eval_model(cfg, net.module)
    logger.info(tabulate([mious, ], headers=heads, tablefmt='orgtbl'))

    return
       
def main():
    torch.cuda.set_device(args.local_rank)
    dist.init_process_group(
        backend='nccl',
        init_method='tcp://127.0.0.1:{}'.format(args.port),
        world_size=torch.cuda.device_count(),
        rank=args.local_rank
    )
    if not osp.exists(cfg.respth): os.makedirs(cfg.respth)
    setup_logger(f'{cfg.model_type}-{cfg.dataset.lower()}-train', cfg.respth)
    train()


if __name__ == "__main__":
    main()

In [4]:
from PIL import Image, ImageDraw as D

In [7]:
i=Image.open("/home/meiyih/Attention-Guided-Low-light-Image-Enhancement-with-Scene-Text-Restoration/dataset/SID/Sony/test/long/10011_00_10s.png")

In [8]:
draw=D.Draw(i)
draw.rectangle([(1930,2122),(1987,2155)],outline="white")


In [9]:
draw=D.Draw(i)
draw.rectangle([(2202,2570),(2351,2821)],outline="white")

In [10]:
draw=D.Draw(i)
draw.rectangle([(2156,2689),(2259,2813)],outline="white")

In [11]:
draw=D.Draw(i)
draw.rectangle([(2478,20),(2702,37)],outline="white")

In [12]:
draw=D.Draw(i)
draw.rectangle([(2598,86),(2676,116)],outline="white")

In [13]:
draw=D.Draw(i)
draw.rectangle([(2523,94),(2598,118)],outline="white")

In [14]:
i.show()